# Testing different triangulation (mesh generation) methods

In [ ]:
import numpy as np
import os
import open3d as o3d
from scipy.spatial import Delaunay

import matplotlib.pyplot as plt

In [ ]:
# Random points in 2D
N_pts = 1000
P = np.random.rand(N_pts, 2)

In [ ]:
# Read in point cloud data
pcfile = os.path.join(os.getcwd(),'..', 'data', 'airsim', 'blocks_0.pcd')
pcd = o3d.io.read_point_cloud(pcfile)
P = np.asarray(pcd.points)

In [ ]:
thetas = np.arctan2(P[:,1], P[:,0])
Rxy = np.sqrt(P[:,0]**2 + P[:,1]**2)
phis = np.arctan2(P[:,2], Rxy)

In [ ]:
fig = plt.figure(figsize=(12,10))
ax = fig.add_subplot(1, 1, 1)
ax.scatter(thetas, phis, s=4)

In [ ]:
DT = Delaunay(np.stack((thetas,phis), axis=1))

Plotly Mesh3d

In [ ]:
import plotly.graph_objects as go

In [ ]:
# Read in point cloud data
pcfile = os.path.join(os.getcwd(),'..', 'data', 'airsim', 'blocks_0.pcd')
pcd = o3d.io.read_point_cloud(pcfile)
P = np.asarray(pcd.points)